In [ ]:
import cv2
import numpy as np


In [ ]:

# Paths to the YOLO model files
config_path = "yolov3.cfg"  # Replace with your config file path
weights_path = "yolov3.weights"  # Replace with your weights file path
names_path = "coco.names"  # Replace with your class names file path


In [ ]:

# Load class names
with open(names_path, "r") as f:
    class_names = [line.strip() for line in f.readlines()]


In [ ]:

# Load YOLO model
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Get the output layer names
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]


In [ ]:

# Load an image
image = cv2.imread("image.jpg")  # Replace with your image path
height, width = image.shape[:2]

# Prepare the image for the model
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)

# Perform forward pass
outputs = net.forward(output_layers)


In [ ]:

# Initialize lists for detection results
boxes = []
confidences = []
class_ids = []

# Process the outputs
for output in outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        # Filter detections by confidence threshold
        if confidence > 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply non-max suppression
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)


In [ ]:

# Draw the results
for i in indices.flatten():
    x, y, w, h = boxes[i]
    label = f"{class_names[class_ids[i]]}: {confidences[i]:.2f}"
    color = (0, 255, 0)

    cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Display the image
cv2.imshow("YOLO Object Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
